In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
import metal
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
from dataset import STSBDataset

In [4]:
from BERT_tasks import create_task
tasks = [create_task('STSB')]

Loading STSB


100%|██████████| 1500/1500 [00:00<00:00, 1608.65it/s]


In [5]:
tasks[0].data_loaders['train'] = tasks[0].data_loaders['valid']

In [ ]:
%%time

from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model,
    tasks,
    n_epochs=3,
    lr=0.0001, l2=0,
    progress_bar=True,
    log_every=50,
    score_every=100,
    log_unit="batches",
    checkpoint_best=True,
    checkpoint_metric="STSB/valid/spearman_corr",
    checkpoint_metric_mode="max",
)

[ (0.03 epo)]: TRAIN:[loss=0.183]
[ (0.05 epo)]: TRAIN:[loss=0.109] VALID:[STSB/pearson_corr=0.0059913634322583675, STSB/spearman_corr=0.006]
Saving model at iteration 100 with best (max) score 0.006
[ (0.08 epo)]: TRAIN:[loss=0.114]
[ (0.10 epo)]: TRAIN:[loss=0.109] VALID:[STSB/pearson_corr=0.0016644200077280402, STSB/spearman_corr=-0.021]
[ (0.13 epo)]: TRAIN:[loss=0.101]
[ (0.16 epo)]: TRAIN:[loss=0.120] VALID:[STSB/pearson_corr=-0.009348052553832531, STSB/spearman_corr=0.007]
Saving model at iteration 300 with best (max) score 0.007
[ (0.18 epo)]: TRAIN:[loss=0.134]
[ (0.21 epo)]: TRAIN:[loss=0.138] VALID:[STSB/pearson_corr=0.0012311202008277178, STSB/spearman_corr=0.024]
Saving model at iteration 400 with best (max) score 0.024
[ (0.23 epo)]: TRAIN:[loss=0.095]
[ (0.26 epo)]: TRAIN:[loss=0.127] VALID:[STSB/pearson_corr=-0.010563547722995281, STSB/spearman_corr=0.025]
Saving model at iteration 500 with best (max) score 0.025
[ (0.29 epo)]: TRAIN:[loss=0.123]


[ (0.31 epo)]: TRAIN:[loss=0.105] VALID:[STSB/pearson_corr=-0.009670714847743511, STSB/spearman_corr=-0.017]
[ (0.34 epo)]: TRAIN:[loss=0.109]
[ (0.37 epo)]: TRAIN:[loss=0.128] VALID:[STSB/pearson_corr=0.03394509106874466, STSB/spearman_corr=0.049]
Saving model at iteration 700 with best (max) score 0.049
[ (0.39 epo)]: TRAIN:[loss=0.117]
